### Current Location to Event Hub

In [2]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
from azure.eventhub.exceptions import EventHubError
import pandas as pd
import json
import pandas as pd
from datetime import datetime
import random
import geopandas as gpd
import smtplib
from azure.eventhub import EventHubConsumerClient
import time
from geopy.distance import geodesic
from datetime import datetime, timedelta
import random
import asyncio
import json
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
import pydoc

In [2]:
from geopy.geocoders import GoogleV3

# Initialize the geocoder
geolocator = GoogleV3(api_key='xxxxxxxxxxxxx')

# Specify the location you want to get coordinates for
location = "Estádio da Luz, Lisboa"

# Geocode the location
location_data = geolocator.geocode(location)

# Extract the latitude and longitude
latitude = location_data.latitude
longitude = location_data.longitude

# Print the coordinates
print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")


Latitude: 38.7526905
Longitude: -9.184692


# Send data of different users for the UseCase 2

## 10 different locations

In [4]:
# Replace these with your actual Azure Event Hub credentials
EVENT_HUB_CONNECTION_STRING = 'xxxxxxx'
EVENT_HUB_NAME  = 'python-location'


def generate_random_location(fan_id):
    # Lisbon latitude and longitude range
    lisbon_latitude_range = (38.700045, 38.805514)
    lisbon_longitude_range = (-9.211691, -9.124352)

    latitude = random.uniform(*lisbon_latitude_range)
    longitude = random.uniform(*lisbon_longitude_range)

    return {"fan_id": fan_id, "latitude": latitude, "longitude": longitude}


async def send_event_to_eventhub(producer, location):
    fan_id = location["fan_id"]
    latitude = location["latitude"]
    longitude = location["longitude"]

    # Format the event data as JSON
    event_data = json.dumps({
        "fan_id": fan_id,
        "latitude": latitude,
        "longitude": longitude
    })

    # Send the event data as bytes
    await producer.send_batch([EventData(body=event_data.encode("utf-8"))])
    print(f"Sent event for Fan ID {fan_id}")


async def simulate_fan_locations(duration=60, num_fans=10):
    producer = EventHubProducerClient.from_connection_string(EVENT_HUB_CONNECTION_STRING)

    fan_ids = random.sample(range(1, num_fans + 1), num_fans)

    try:
        async with producer:
            tasks = []
            for fan_id in fan_ids:
                location = generate_random_location(fan_id)
                tasks.append(send_event_to_eventhub(producer, location))

            # Send location updates for the specified duration
            end_time = asyncio.get_event_loop().time() + duration
            while asyncio.get_event_loop().time() < end_time:
                await asyncio.gather(*tasks)
                await asyncio.sleep(1)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        await producer.close()

def main():
    loop = asyncio.get_event_loop()
    if loop.is_running():
        loop.create_task(simulate_fan_locations(duration=60, num_fans=10))
    else:
        loop.run_until_complete(simulate_fan_locations(duration=60, num_fans=10))
        loop.run_until_complete(loop.shutdown_asyncgens())
        loop.close()

if __name__ == "__main__":
    main()


Sent event for Fan ID 5
Sent event for Fan ID 10
Sent event for Fan ID 8
Sent event for Fan ID 1
Sent event for Fan ID 9
Sent event for Fan ID 2
Sent event for Fan ID 6
Sent event for Fan ID 4
Sent event for Fan ID 7
Sent event for Fan ID 3
An error occurred: cannot reuse already awaited coroutine


## Read Data From Event Hub and Send Notification (User Case 2)

### Event hub -> Stream Analytics -> SQL Database -> Python

### Email based on distance and time to the match

In [6]:
import math
import pyodbc
from datetime import datetime, timedelta
import smtplib


# Haversine formula to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert coordinates to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Calculate distance using Haversine formula
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

# Database connection details
server = 'thesis-sql-server.database.windows.net'
database = 'Thesis-asql'
username = 'spereira'
password = 'xxxxxxxx'

# Connect to the SQL Server database
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER=thesis-sql-server.database.windows.net;DATABASE=Thesis-asql;UID=spereira;PWD=xxxxxxxx'
)
cursor = conn.cursor()

# Execute an SQL query to retrieve data
cursor.execute("""  
    SELECT TOP 1 a.fan_id, a.name, a.email, b.Latitude, b.Longitude, b.EventEnqueuedUtcTime
    FROM fans a
    LEFT JOIN [FanLocations] b ON a.fan_id = b.fan_ID
    WHERE a.fan_id = 1
    ORDER BY b.EventEnqueuedUtcTime DESC 
""")

data = cursor.fetchone()

# Close the database connection
conn.close()

# Reference point coordinates (Estádio da Luz)
reference_latitude = 38.752711
reference_longitude = -9.184773999999999

# Check distances and send personalized messages
fan_id, name, email, latitude, longitude, event_time = data
distance = calculate_distance(latitude, longitude, reference_latitude, reference_longitude)

message = ""
if distance > 5:
    message = f"Dear {name},\n\nWe noticed that you are still far away from the stadium, and the kick-off is near. We advise you to start heading to the stadium as soon as possible due to intense traffic.\n\nRegards,\nYour Sports Team"
else:
    message = f"Dear {name},\n\nSince you are nearby the stadium, come visit our official store and enjoy 20% off on every article before the game starts.\n\nRegards,\nYour Sports Team"

       
    
# Email configuration
smtp_server = 'smtp.office365.com'
smtp_port = 587
smtp_username = 'sender@gmail.com'
smtp_password = 'xxxxxxxxxxxx'
sender_email = 'sender@gmail.com'



current_time = datetime.now()
match_time = datetime(current_time.year, current_time.month, 21, 18, 0)  # Set the desired match time
time_difference = match_time - current_time

if time_difference <= timedelta(hours=3.0):
    # Compose the email body with personalized messages
    subject = 'Information ahead of the game'
    body = message

    # Compose the email
    email_message = f'Subject: {subject}\n\n{body}'


    # Send the personalized email to the fan
    if email:
        receiver_email = email

        # Compose the email body with personalized messages
        subject = 'Information ahead of the game'
        body = message

        # Compose the email
        email_message = f'Subject: {subject}\n\n{body}'

        # Send the email
        try:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()
                server.login(smtp_username, smtp_password)
                server.sendmail(sender_email, receiver_email, email_message)
            print(f'Email sent successfully to {name}!')
        except Exception as e:
            print(f'An error occurred while sending the email: {str(e)}')
    else:
        print('Fan email not available. Cannot send personalized email.')


Email sent successfully to John Smith!


## USER CASE 1

### Email based on if the fan has a ticket or not 

In [7]:
import smtplib
import pyodbc
from datetime import datetime, timedelta

# Database connection details
server = 'thesis-sql-server.database.windows.net'
database = 'xxxxxxxxxxx'
username = 'spereira'
password = 'xxxxxxxxxxxxxxxx'

# Connect to the SQL Server database
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER=thesis-sql-server.database.windows.net;DATABASE=xxxxxxxxx;UID=spereira;PWD=xxxxxxxxxxx'
)
cursor = conn.cursor()

# Email details
smtp_server = 'smtp.office365.com'
smtp_port = 587
smtp_username = 'sender@gmail.com'
smtp_password = 'xxxxxxxxxxxx'
sender_email = 'sender@gmail.com'
receiver_email = 'simao.er.costa.pereira@gmail.com'

def send_email(recipient_email):
    subject = 'Ticket Reminder'
    body = 'Dear fan,\n\nThis is a reminder that you do not have a ticket for the upcoming match. Please visit our website to get your hands on the last avaialable tickets.\n\nRegards,\nYour Sports Team'

    message = f'Subject: {subject}\n\n{body}'

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, smtp_password)
        server.sendmail(sender_email, recipient_email, message)


# try:
    
#     # Get the current datetime
#     current_time = datetime.now()

#     # Calculate the game start time
#     game_start_time = datetime(current_time.year, current_time.month, current_time.day, 18, 0)  # Assuming the game starts at 6pm

#     # Calculate the time difference in hours
#     time_difference_hours = (game_start_time - current_time).total_seconds() / 3600
    
    
try:
    # Get the current datetime
    current_time = datetime.now()

    # Query the game start time from the matches table
    with conn.cursor() as cursor:
        query = "SELECT game_date, game_time FROM Matches WHERE match_id = 1"
        cursor.execute(query)
        result = cursor.fetchone()
        game_date, game_time = result[0], result[1]

    # Combine the date and time to create the game start time
    game_start_time = datetime.combine(game_date, game_time)

    # Calculate the time difference in hours
    time_difference_hours = (game_start_time - current_time).total_seconds() / 3600

# except Exception as e:
#     print("An error occurred:", str(e))



# Check if the game starts in 5 or fewer hours --- CHECK THE START TIME IN THE SQL TABLE. IT NEEDS TO BE CHANGED WITH TIME FOR THIS TO WORK
    if time_difference_hours <= 5:
        # Check if the stadium has at least one unoccupied seat
        with conn.cursor() as cursor:
            query = "SELECT COUNT(*) FROM stadium_occupancy WHERE is_occupied = 0"
            cursor.execute(query)
            result = cursor.fetchone()
            unoccupied_count = result[0]

        if unoccupied_count > 0:
            # Create a cursor object
            with conn.cursor() as cursor:
                # Execute the query
                query = """
                
                SELECT name, email FROM fan_details WHERE has_ticket = 0 and match_id = 1
           
                """
                
                cursor.execute(query)

                # Fetch all the rows
                rows = cursor.fetchall()

                # Iterate over the rows
                for row in rows:
                    name, email = row

                    # Send an email to fans without a ticket
                    send_email(email)
                    print(f"Email sent to {name} at {email}")
        else:
            print("No unoccupied seats in the stadium.")
    else:
        print("Current time is not within the specified cutoff period.")




finally:
     # Close the database connection
        conn.close()



Email sent to Jane Doe at simao.er.costa.pereira@gmail.com
Email sent to Michael Johnson at simao.er.costa.pereira@gmail.com
Email sent to Sarah Wilson at simao.er.costa.pereira@gmail.com
Email sent to David Lee at simao.er.costa.pereira@gmail.com
Email sent to Emily Davis at simao.er.costa.pereira@gmail.com


## Marketing email at half time 

In [8]:
import smtplib
import pyodbc
from datetime import datetime, timedelta

# Database connection details
server = 'thesis-sql-server.database.windows.net'
database = 'xxxxxx'
username = 'xxxxxxx'
password = 'xxxxxx'

# Connect to the SQL Server database
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER=thesis-sql-server.database.windows.net;DATABASE=xxxxxx-xxx;UID=spereira;PWD=xxxxxxx'
)
cursor = conn.cursor()

# Email details
smtp_server = 'smtp.office365.com'
smtp_port = 587
smtp_username = 'sender@gmail.com'
smtp_password = 'xxxxxx'
sender_email = 'sender@gmail.com'
receiver_email = 'simao.er.costa.pereira@gmail.com'

def send_email(recipient_email):
    subject = 'Halftime Promotions'
    body = 'Dear fan,\n\nEnjoy our halftime discounts in food, beverages and merchadising next to your entrance gate.  .\n\nRegards,\nYour Sports Team'

    message = f'Subject: {subject}\n\n{body}'

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, smtp_password)
        server.sendmail(sender_email, recipient_email, message)

    
    
try:
    # Get the current datetime
    current_time = datetime.now()

    # Query the game start time from the matches table
    with conn.cursor() as cursor:
        query = "SELECT game_date, game_time FROM Matches WHERE match_id = 1"
        cursor.execute(query)
        result = cursor.fetchone()
        game_date, game_time = result[0], result[1]

    # Combine the date and time to create the game start time
    #game_start_time = datetime.combine(game_date, game_time)
    game_start_time = datetime(2023, 7, 14, 17, 0)

    # Calculate halftime
    halftime = (game_start_time + timedelta(minutes=45))

# except Exception as e:
#     print("An error occurred:", str(e))



# Check if the game is at halftime
    if current_time >= halftime:
        # Check if the stadium has at least one unoccupied seat
        with conn.cursor() as cursor:
            query = "SELECT COUNT(*) FROM stadium_occupancy WHERE is_occupied = 1"
            cursor.execute(query)
            result = cursor.fetchone()
            unoccupied_count = result[0]

        if unoccupied_count > 0:
            # Create a cursor object
            with conn.cursor() as cursor:
                # Execute the query
                query = """
                
                SELECT name, email FROM fan_details WHERE has_ticket = 1 and match_id = 1
           
                """
                
                cursor.execute(query)

                # Fetch all the rows
                rows = cursor.fetchall()

                # Iterate over the rows
                for row in rows:
                    name, email = row

                    # Send an email to fans without a ticket
                    send_email(email)
                    print(f"Email sent to {name} at {email}")
        else:
            print("No unoccupied seats in the stadium.")
    else:
        print("Current time is not within the specified cutoff period.")




finally:
     # Close the database connection
        conn.close()



Email sent to John Smith at simao.er.costa.pereira@gmail.com
Email sent to Robert Thompson at simao.er.costa.pereira@gmail.com
Email sent to Jennifer Brown at simao.er.costa.pereira@gmail.com
Email sent to Christopher Martinez at simao.er.costa.pereira@gmail.com
Email sent to Jessica Taylor at simao.er.costa.pereira@gmail.com
